In [1]:
%matplotlib inline
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
import random

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

from skimage.feature import greycomatrix, greycoprops

In [2]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [3]:
TRAINING_SIZE = 50
VALIDATION_SIZE = 5  # Size of the validation set.
TEST_SIZE = 20
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 128 # 64
NUM_EPOCHS = 200
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 10

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!

PATCH_SIZE = 16
EXTRA_CONTEXT = 0
CONTEXT_SIZE = EXTRA_CONTEXT + PATCH_SIZE + EXTRA_CONTEXT

In [4]:
random.seed(SEED)

In [5]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 
test_dir = "test/"

In [6]:
# returns all train images in rgb
def get_train_imgs():
    imgs = []
    for i in range(TRAINING_SIZE):
        imageid = "satImage_%.3d" % (i+1)
        image_filename = train_data_filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            raise Exception('File ' + image_filename + ' does not exist')
    return np.asarray(imgs)

# returns all test images as rgb
def get_test_imgs():
    imgs = []
    for i in range(TEST_SIZE):
        imagename = "test_" + str(i+1)
        image_filename = test_dir + imagename + "/" + imagename + ".png"
        if os.path.isfile(image_filename):
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            raise Exception('File ' + image_filename + ' does not exist')
    return np.asarray(imgs)

# Get prediction overlaid on the original image for given input image
def get_prediction_with_overlay(img):
    
    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img[:,:,:3], img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(img):
    
    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img[:,:,:3], img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

# returns a numpy array [Saturation, Lightness] of a pixel
def sat_light(rgb):
    Cmax = rgb.max()
    Cmin = rgb.min()
    d = Cmax - Cmin
    L = (Cmax + Cmin) / 2
    S = d / (1 - (abs(2 * L - 1)))
    if np.isnan(S):
        S = 0
    return np.asarray([S, L], dtype=np.float32)

#returns edges
def texture_features(image):
    result = np.empty(list(image.shape) + [2])
    h = image.shape[0]
    w = image.shape[1]
    result[:,:,0] = np.vstack((image[:-1,:] - image[1:,:], np.zeros((1,w))))
    result[:,:,1] = np.hstack((image[:,:-1] - image[:,1:], np.zeros((h,1))))
    return np.max(np.abs(result),axis=2)

NUM_CHANNELS = 6 # RGB, Saturation, Lightness, Edges
def preprocess(img):
    new_shape = list(img.shape)
    new_shape[2] = NUM_CHANNELS
    result = np.empty(new_shape, dtype=np.float32)
    result[:,:,:3] = img
    for y in range(len(img)):
        for x in range(len(img[y])):
            result[y,x,3:5] = sat_light(result[y,x,:3])
    #print('Saturation and lightness computed.')
    result[:,:,5] = texture_features(result[:,:,4])
    #print('Texture deatures computed.')
    return result

# takes an array of images, preprocesses them and stores the result
# in save_dir/preprocessed.npy
def preprocess_imgs(imgs, save_dir):
    assert save_dir.endswith('/')
    processed = []
    for i in range(len(imgs)):
        processed.append(preprocess(imgs[i]))
        print('Image ' + str(i) + ' preprocessed.')
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    processed_np = np.asarray(processed)
    np.save(save_dir + 'preprocessed2.npy', processed_np)
    return processed_np

def load_preproc(save_dir):
    assert save_dir.endswith('/')
    return np.load(save_dir + 'preprocessed2.npy')

def load_labels():
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, TRAINING_SIZE+1):
        imageid = "satImage_%.3d" % i
        image_filename = train_labels_filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    labels = np.asarray(gt_imgs, dtype=np.float32)
    return labels

def store_channel_to_img(chan):
    img = (chan * 255).astype('uint8')
    png = Image.fromarray(img)
    png.show
    return png

def show(chan):
    chan = chan + chan.min()
    chan = chan / chan.max()
    img = (chan * 255).astype('uint8')
    png = Image.fromarray(img)
    png.show
    return png

In [7]:
PREPROCESS_TRAIN_DATA = False
PREPROCESS_TEST_DATA = False

if PREPROCESS_TRAIN_DATA:
    preprocess_imgs(get_train_imgs(), train_data_filename)
if PREPROCESS_TEST_DATA:
    preprocess_imgs(get_test_imgs(), test_dir)

In [8]:
train_data = load_preproc(train_data_filename)
train_labels = load_labels()
print(train_data.shape)
print(train_labels.shape)

(100, 400, 400, 6)
(50, 400, 400)


In [9]:
# center data
train_data = train_data * 2 - 1

In [10]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, CONTEXT_SIZE, CONTEXT_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')

In [11]:
FILTER1_SIZE = 5
FILTER1_COUNT = 16
FILTER1_STRIDES = [1,1,1,1]

POOL1_FACTOR = 3

FILTER2_SIZE = 5
FILTER2_COUNT = 32
FILTER2_STRIDES = [1,1,1,1]

POOL2_FACTOR = 5

FILTER3_SIZE = 3
FILTER3_COUNT = 56
FILTER3_STRIDES = [1,1,1,1]

POOL3_FACTOR = 9

FILTER4_SIZE = 3
FILTER4_COUNT = 64
FILTER4_STRIDES = [1,1,1,1]

POOL4_FACTOR = 1

FC1_SIZE = 256
FC2_SIZE = 256

In [12]:
filter1_shape = [FILTER1_SIZE, FILTER1_SIZE, NUM_CHANNELS, FILTER1_COUNT]
filter1 = tf.Variable(tf.truncated_normal(filter1_shape, stddev=.1, seed=SEED))
conv1_biases = tf.Variable(tf.zeros(shape=[FILTER1_COUNT]))

filter2_shape = [FILTER2_SIZE, FILTER2_SIZE, FILTER1_COUNT, FILTER2_COUNT]
filter2 = tf.Variable(tf.truncated_normal(filter2_shape, stddev=.1, seed=SEED))
conv2_biases = tf.Variable(tf.zeros(shape=[FILTER2_COUNT]))

filter3_shape = [FILTER3_SIZE, FILTER3_SIZE, FILTER2_COUNT, FILTER3_COUNT]
filter3 = tf.Variable(tf.truncated_normal(filter3_shape, stddev=.1, seed=SEED))
conv3_biases = tf.Variable(tf.zeros(shape=[FILTER3_COUNT]))

filter4_shape = [FILTER4_SIZE, FILTER4_SIZE, FILTER3_COUNT, FILTER4_COUNT]
filter4 = tf.Variable(tf.truncated_normal(filter4_shape, stddev=.1, seed=SEED))
conv4_biases = tf.Variable(tf.zeros(shape=[FILTER4_COUNT]))

side = CONTEXT_SIZE
reshape_depth = side * side * FILTER4_COUNT
fc1_weights = tf.Variable(
    tf.truncated_normal([reshape_depth, FC1_SIZE], stddev=.1, seed=SEED))
fc1_biases = tf.Variable(tf.zeros(shape=[FC1_SIZE]))

fc2_weights = tf.Variable(
    tf.truncated_normal([FC1_SIZE, FC2_SIZE], stddev=.1, seed=SEED))
fc2_biases = tf.Variable(tf.zeros(shape=[FC2_SIZE]))

fc_last_weights = tf.Variable(
    tf.truncated_normal([FC2_SIZE, cil.NUM_LABELS], stddev=.1, seed=SEED))
fc_last_biases = tf.Variable(tf.zeros(shape=[cil.NUM_LABELS]))

In [13]:
def model(input_node, dropout=False):
    print(input_node.get_shape())

    conv1 = tf.nn.conv2d(input_node, filter1, FILTER1_STRIDES, 
                         padding='SAME', name='conv1')

    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    pool1 = tf.nn.max_pool(relu1, ksize=[1,POOL1_FACTOR,POOL1_FACTOR,1], 
                           strides=[1,1,1,1], padding='SAME')
    print(pool1.get_shape())
    ######################
    conv2 = tf.nn.conv2d(pool1, filter2, FILTER2_STRIDES, 
                         padding='SAME', name='conv2')

    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    pool2 = tf.nn.max_pool(relu2, ksize=[1,POOL2_FACTOR,POOL2_FACTOR,1], 
                           strides=[1,1,1,1], padding='SAME')
    print(pool2.get_shape())
    ######################
    conv3 = tf.nn.conv2d(pool2, filter3, FILTER3_STRIDES, 
                         padding='SAME', name='conv2')

    relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))

    pool3 = tf.nn.max_pool(relu3, ksize=[1,POOL3_FACTOR,POOL3_FACTOR,1], 
                           strides=[1,1,1,1], padding='SAME')
    print(pool3.get_shape())
    ######################
    conv4 = tf.nn.conv2d(pool3, filter4, FILTER4_STRIDES, 
                         padding='SAME', name='conv2')

    relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))

    pool4 = tf.nn.max_pool(relu4, ksize=[1,POOL4_FACTOR,POOL4_FACTOR,1], 
                           strides=[1,1,1,1], padding='SAME')
    print(pool4.get_shape())
    ######################
    shp = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shp[0], shp[1] * shp[2] * shp[3]])
    print(reshape.get_shape())

    fc1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if (dropout):
        fc1 = tf.nn.dropout(fc1, .5, seed=SEED)
    print(fc1.get_shape())

    fc2 = tf.nn.relu(tf.matmul(fc1, fc2_weights) + fc2_biases)
    if (dropout):
        fc2 = tf.nn.dropout(fc2, .5, seed=SEED)
    print(fc2.get_shape())

    fc_last = tf.matmul(fc2, fc_last_weights) + fc_last_biases
    print(fc_last.get_shape())
    return fc_last

In [14]:
logits = model(train_data_node, dropout=True)

(128, 16, 16, 6)
(128, 16, 16, 16)
(128, 16, 16, 32)
(128, 16, 16, 56)
(128, 16, 16, 64)
(128, 16384)
(128, 256)
(128, 256)
(128, 2)


In [15]:
output = tf.nn.softmax(logits)

In [16]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, train_labels_node))
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases) +
                tf.nn.l2_loss(fc_last_weights) + tf.nn.l2_loss(fc_last_biases))
#loss += 5e-4 * regularizers

In [17]:
DECAY_STEP = 100
batch_idx = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    1e-6,                # Base learning rate.
    batch_idx * BATCH_SIZE,  # Current index into the dataset.
    DECAY_STEP,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)

In [18]:
#optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
#    loss, global_step=batch_idx)
opt = tf.train.MomentumOptimizer(learning_rate, 0.9)
optimizer = opt.minimize(loss, global_step=batch_idx)

In [19]:
def get_window(image, y, x, truth=None):
    #padding the image with wrap-around
    window = np.take(np.take(image, 
                     range(y,y+CONTEXT_SIZE), axis=0,mode='wrap'), 
                     range(x,x+CONTEXT_SIZE), axis=1,mode='wrap')
    if not truth is None:
        truth_window = np.take(np.take(truth,
     range(y+EXTRA_CONTEXT,y+EXTRA_CONTEXT+PATCH_SIZE), axis=0,mode='wrap'),
     range(x+EXTRA_CONTEXT,x+EXTRA_CONTEXT+PATCH_SIZE), axis=1,mode='wrap')
        return (window, truth_window)
    else:
        return window

def get_training_samples(batch_size):    
    batch = np.empty((batch_size, CONTEXT_SIZE, CONTEXT_SIZE, NUM_CHANNELS))
    labels = np.empty((batch_size, cil.NUM_LABELS))
    for i in range(batch_size):
        pic_idx = random.randrange(0,TRAINING_SIZE)
        pic = train_data[pic_idx]
        y = random.randrange(0,pic.shape[0])
        x = random.randrange(0,pic.shape[1])
        
        window, truth = get_window(pic, y, x, train_labels[pic_idx])
        label = truth.mean()
        batch[i] = window
        labels[i][0] = label
        labels[i][1] = 1 - label
    return (batch, labels)

In [20]:
(img,lbl) = get_training_samples(1)

In [41]:
show(img[0,:,:,3:]/2+.5)

In [22]:
s = tf.Session()

In [23]:
s.run(tf.initialize_all_variables())
print ('Initialized!')

Initialized!


In [24]:
batch, labels = get_training_samples(BATCH_SIZE)

feed_dict = { train_data_node: batch
                    , train_labels_node: labels}

In [26]:
s.run(optimizer, feed_dict=feed_dict)

In [39]:
for (grad,var) in opt.compute_gradients(loss):
    if grad is not None:
        grads = grad.eval(session=s, feed_dict=feed_dict)
        i = grads.argmax()
        print('Grad: ' + str(grads.flat[i]))
        val = var.eval(session=s).flat[i]
        print('Var: ' + str(val))
        print('Ratio: ' + str(grads.flat[i]/val))
        print('\n')

Grad: 1.29971
Var: 0.168956
Ratio: 7.69259


Grad: 14.1659
Var: -1.53676e-05
Ratio: -921808.0


Grad: 9.24941
Var: 0.019656
Ratio: 470.565


Grad: 6.91624
Var: -7.00364e-06
Ratio: -987521.0


Grad: 6.5277
Var: 0.0953063
Ratio: 68.4918


Grad: 3.45631
Var: -3.28297e-06
Ratio: -1.0528e+06


Grad: 9.96509
Var: 0.0542281
Ratio: 183.762


Grad: 2.78218
Var: -2.44939e-06
Ratio: -1.13586e+06


Grad: 2.20407
Var: -0.0577689
Ratio: -38.1532


Grad: 0.211232
Var: -1.78738e-07
Ratio: -1.1818e+06


Grad: 17.675
Var: 0.151721
Ratio: 116.496


Grad: 0.165171
Var: -1.19327e-07
Ratio: -1.38418e+06


Grad: 52.4268
Var: 0.183034
Ratio: 286.432


Grad: 0.490957
Var: -5.33685e-07
Ratio: -919937.0




In [ ]:
losses = []
for epoch in range(NUM_EPOCHS):
    for i in range(RECORDING_STEP):
        batch, labels = get_training_samples(BATCH_SIZE)

        feed_dict = { train_data_node: batch
                    , train_labels_node: labels}
        
        s.run(optimizer, feed_dict=feed_dict)

    batch, labels = get_training_samples(BATCH_SIZE)
    feed_dict = { train_data_node: batch
                , train_labels_node: labels}
    l, pred = s.run([loss, output], feed_dict=feed_dict)
    error_rate = cil.error_rate(pred,labels)
    print('Epoch ' + str(epoch) + ':')
    print('Loss: ' + str(l))
    print('Error: ' + str(error_rate))
    losses.append(l)

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
pred

In [ ]:
pred_input = tf.placeholder(
    tf.float32,
    shape=(1, CONTEXT_SIZE, CONTEXT_SIZE, NUM_CHANNELS),
    name='pred_input')
pred_output = tf.nn.softmax(model(pred_input))

In [ ]:
imgs = train_data[:1]
# assuming images are quadratic
num_patches = imgs[0].shape[0] // PATCH_SIZE
truth = np.empty((len(imgs), num_patches, num_patches, cil.NUM_LABELS))
preds = np.empty((len(imgs), num_patches, num_patches, cil.NUM_LABELS))
for img_idx in range(len(imgs)):
    img = imgs[img_idx]
    for patch_y in range(num_patches):
        y = patch_y * PATCH_SIZE - EXTRA_CONTEXT
        for patch_x in range(num_patches):
            x = patch_x * PATCH_SIZE - EXTRA_CONTEXT
            window, window_truth = get_window(img, y, x, train_labels[img_idx])
            pred = s.run(pred_output, {pred_input: [window]})
            preds[img_idx, patch_y, patch_x] = pred
            label = window_truth.mean()
            truth[img_idx, patch_y, patch_x, 0] = label
            truth[img_idx, patch_y, patch_x, 1] = 1 - label
    print('Done with image ' + str(img_idx))
truth = truth.reshape((len(imgs) * num_patches * num_patches, cil.NUM_LABELS))
preds = preds.reshape((len(imgs) * num_patches * num_patches, cil.NUM_LABELS))
error_rate = cil.error_rate(preds, truth)
print(error_rate)

In [ ]:
preds

In [ ]:
preds_m = np.empty(preds.shape)

In [ ]:
preds_m[:,0] = preds[:,0]
preds_m[:,1] = 1 - preds_m[:,0]
cil.error_rate(preds_m, truth)

In [ ]:
label_img = cil.label_to_img(imgs[0].shape[0],imgs[0].shape[1],
                             PATCH_SIZE,PATCH_SIZE,preds_m)
show(label_img)

In [ ]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
    
for i in range(TRAINING_SIZE):
    pimg = get_prediction_with_groundtruth(train_preproc[i])
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i+1) + ".png")
    oimg = get_prediction_with_overlay(train_preproc[i])
    oimg.save(prediction_training_dir + "overlay-8x8_" + str(i+1) + ".png")  

In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)

test_preproc = load_preproc(test_dir)
for i in range(TEST_SIZE):
    pred = get_prediction(test_preproc[i])
    pimg = cil.img_float_to_uint8(pred)
    Image.fromarray(pimg).save(prediction_test_dir + "prediction_" + str(i+1) + ".png")
    oimg = cil.make_img_overlay(test_preproc[i], pred)
    oimg.save(prediction_test_dir + "overlay-8x8_" + str(i+1) + ".png")

In [ ]:
import re
th = 0.75

In [ ]:
# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    image_filenames = []
    for i in range(1, 51):
        imagename = 'prediction_' + str(i)
        image_filename = 'predictions_test/' + imagename + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)
    
save('submission_lukas_11_2.5.csv')